### More OpenAI Models
Reference: https://platform.openai.com/docs/models

In [ ]:
import os

from dotenv import load_dotenv

from openai import OpenAI, AzureOpenAI

In [ ]:
load_dotenv()

In [ ]:
client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

#### DALL·E (文字生圖片)

DALL·E is a AI system that can create realistic images and art from a description in natural language.  
Reference: https://platform.openai.com/docs/models/dall-e

In [ ]:
prompt = '一個有著狐狸耳朵的少女，色彩豐富的、動漫畫風'

completion = client.images.generate(
    prompt=prompt,
    model='dall-e-3',
    n=1,
    quality='hd',
    size='1024x1024',
    style='natural',
)

image_url = completion.data[0].url
image_url

#### Whisper (影音轉字幕)

Whisper is a general-purpose speech recognition model.  
Reference: https://platform.openai.com/docs/models/whisper

In [ ]:
from pytubefix import YouTube

# Download the video
url = 'https://www.youtube.com/watch?v=vgYi3Wr7v_g'
yt = YouTube(url)
ys = yt.streams.get_highest_resolution()
audio_filename = yt.title
audio_filepath = ys.download(output_path=f'../data')
audio_filepath

In [ ]:
with open(audio_filepath, "rb") as file:
    transcription = client.audio.transcriptions.create(
        file=file,
        model="whisper",
        response_format='srt'
    )

subtitle_filepath = f"../data/{audio_filename}.srt"
with open(subtitle_filepath, "w") as file:
    file.write(transcription)

print(transcription)

#### TTS (文本轉音訊)

TTS is an AI model that converts text to natural sounding spoken text.   
Reference: https://platform.openai.com/docs/models/tts

In [ ]:
with open(subtitle_filepath, "r") as f:
    content = f.readlines()

subtitles = ''
for idx in range(2, len(content), 4):
  subtitles += content[idx].strip() + ' '

completion = client.audio.speech.create(
    input=subtitles,
    model="tts-1",
    voice="alloy",
    response_format='mp3'
)

speech_filepath = f"../data/{audio_filename}.mp3"
completion.write_to_file(speech_filepath)
speech_filepath